In [3]:
import os
import pickle
import argparse
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, roc_auc_score

def compute_average_report_across_runs(reports):
    overall_report = {
        'HC': {'precision': [], 'recall': [], 'f1-score': [], 'support': []},
        'PD': {'precision': [], 'recall': [], 'f1-score': [], 'support': []},
        'accuracy': [],
        'macro avg': {'precision': [], 'recall': [], 'f1-score': [], 'support': []},
        'weighted avg': {'precision': [], 'recall': [], 'f1-score': [], 'support': []},
    }

    # Spoiler: It's gonna be inefficient :)

    for report in reports:
        for key in report.keys():
            if key == 'accuracy':
                overall_report[key].append(report[key])
            else:
                for key2 in report[key].keys():
                    overall_report[key][key2].append(report[key][key2])

    for key in overall_report.keys():
            if key == 'accuracy':
                overall_report[key] = f'{round(np.array(overall_report[key]).mean(), 4)}±{round(np.array(overall_report[key]).std(), 4)}'
            else:
                for key2 in report[key].keys():
                    overall_report[key][key2] = f'{round(np.array(overall_report[key][key2]).mean(), 4)}±{round(np.array(overall_report[key][key2]).std(), 4)}'

    # -- just for a more clean output
    overall_report = pd.DataFrame.from_dict(overall_report).T
    overall_report.iloc[2,0] = ''
    overall_report.iloc[2,1] = ''
    overall_report.iloc[2,3] = overall_report.iloc[3,3]

    return overall_report

def get_reports(exps_dir):
    val_reports = []
    test_reports = []
    run_dirs = os.listdir(exps_dir)
    for run_dir in run_dirs:
        val_preds = []
        val_labels = []
        test_preds = []
        test_labels = []

        run_dir_path = os.path.join(exps_dir, run_dir)
        fold_dirs = os.listdir(run_dir_path)
        
        for fold_dir in fold_dirs:
            model_output_dir = os.path.join(run_dir_path, fold_dir, 'model_output')

            # -- validation set
            val_report_path = os.path.join(os.path.join(model_output_dir, 'validation_classification.pkl'))
            with open(val_report_path, 'rb') as f:
                val_model_output = pickle.load(f)

            val_preds += val_model_output['preds']
            val_labels += val_model_output['labels']

            # -- test set
            test_report_path = os.path.join(os.path.join(model_output_dir, 'test_classification.pkl'))
            with open(test_report_path, 'rb') as f:
                test_model_output = pickle.load(f)

            test_preds += test_model_output['preds']
            test_labels += test_model_output['labels']

        # -- computing reports
        val_reports.append(
            classification_report(
                val_labels,
                val_preds,
                target_names=['HC', 'PD'],
                output_dict=True,
            )
        )

        test_reports.append(
            classification_report(
                test_labels,
                test_preds,
                target_names=['HC', 'PD'],
                output_dict=True,
            )
        )

    return val_reports, test_reports


def get_report_perfold(exps_dir):
    """
    获取每个run的每个fold的报告
    
    Args:
        exps_dir: 存储实验结果的目录路径
        
    Returns:
        val_fold_reports: 字典，键为run_dir，值为该run下每个fold的验证集报告列表
        test_fold_reports: 字典，键为run_dir，值为该run下每个fold的测试集报告列表
    """
    val_fold_reports = {}
    test_fold_reports = {}
    run_dirs = sorted(os.listdir(exps_dir))
    
    for run_dir in run_dirs:
        run_dir_path = os.path.join(exps_dir, run_dir)
        fold_dirs = sorted(os.listdir(run_dir_path))
        
        val_fold_reports[run_dir] = []
        test_fold_reports[run_dir] = []
        
        for fold_dir in fold_dirs:
            model_output_dir = os.path.join(run_dir_path, fold_dir, 'model_output')
            
            # -- validation set
            val_report_path = os.path.join(model_output_dir, 'validation_classification.pkl')
            if os.path.exists(val_report_path):
                with open(val_report_path, 'rb') as f:
                    val_model_output = pickle.load(f)
                
                # 提取概率值用于AUC计算
                val_probs = [preds[1] for preds in val_model_output['probs']]
                val_model_output['probs'] = val_probs  # 更新为单一概率值
                
                # 计算AUC
                val_auc = roc_auc_score(val_model_output['labels'], val_probs)
                
                # 计算单个fold的验证集分类报告
                val_fold_report = classification_report(
                    val_model_output['labels'],
                    val_model_output['preds'],
                    target_names=['HC', 'PD'],
                    output_dict=True,
                )
                
                val_fold_reports[run_dir].append({
                    'fold': fold_dir,
                    'report': val_fold_report,
                    'preds': val_model_output['preds'],
                    'labels': val_model_output['labels'],
                    'probs': val_model_output['probs'],  # 添加概率值
                    'auc': val_auc  # 添加AUC
                })
            
            # -- test set
            test_report_path = os.path.join(model_output_dir, 'test_classification.pkl')
            if os.path.exists(test_report_path):
                with open(test_report_path, 'rb') as f:
                    test_model_output = pickle.load(f)
                
                # 提取概率值用于AUC计算
                test_probs = [preds[1] for preds in test_model_output['probs']]
                test_model_output['probs'] = test_probs  # 更新为单一概率值
                
                # 计算AUC
                test_auc = roc_auc_score(test_model_output['labels'], test_probs)
                
                # 计算单个fold的测试集分类报告
                test_fold_report = classification_report(
                    test_model_output['labels'],
                    test_model_output['preds'],
                    target_names=['HC', 'PD'],
                    output_dict=True,
                )
                
                test_fold_reports[run_dir].append({
                    'fold': fold_dir,
                    'report': test_fold_report,
                    'preds': test_model_output['preds'],
                    'labels': test_model_output['labels'],
                    'probs': test_model_output['probs'],  # 添加概率值
                    'auc': test_auc  # 添加AUC
                })
    
    return val_fold_reports, test_fold_reports

def compute_fold_stats(fold_reports):
    """
    计算每个run中所有fold的性能指标的均值和标准差
    
    Args:
        fold_reports: 字典，键为run_dir，值为该run下每个fold的报告列表
        
    Returns:
        fold_stats: 字典，键为run_dir，值为包含accuracy、f1、precision、recall、sensitivity、specificity、auc的均值和标准差的字典
    """
    fold_stats = {}
    
    for run_dir, fold_results in fold_reports.items():
        accuracies = [fold_result['report']['accuracy'] for fold_result in fold_results]
        f1_scores = [fold_result['report']['PD']['f1-score'] for fold_result in fold_results]
        precisions = [fold_result['report']['PD']['precision'] for fold_result in fold_results]
        recalls = [fold_result['report']['PD']['recall'] for fold_result in fold_results]
        
        # 计算灵敏度(Sensitivity)和特异度(Specificity)
        sensitivities = [fold_result['report']['PD']['recall'] for fold_result in fold_results]  # 灵敏度就是PD的召回率
        specificities = [fold_result['report']['HC']['recall'] for fold_result in fold_results]  # 特异度是HC的召回率
        
        # 添加AUC指标
        aucs = [fold_result['auc'] for fold_result in fold_results]
        
        fold_stats[run_dir] = {
            'accuracy': {
                'mean': np.mean(accuracies),
                'std': np.std(accuracies)
            },
            'f1': {
                'mean': np.mean(f1_scores),
                'std': np.std(f1_scores)
            },
            'precision': {
                'mean': np.mean(precisions),
                'std': np.std(precisions)
            },
            'recall': {
                'mean': np.mean(recalls),
                'std': np.std(recalls)
            },
            'sensitivity': {
                'mean': np.mean(sensitivities),
                'std': np.std(sensitivities)
            },
            'specificity': {
                'mean': np.mean(specificities),
                'std': np.std(specificities)
            },
            'auc': {
                'mean': np.mean(aucs),
                'std': np.std(aucs)
            }
        }
    
    return fold_stats



In [4]:

exps_dir = "/home/yzhong/gits/interpretable-pd/exps/gita-splitmono//cross_full/MONOLOGUE/"
# 获取每个fold的报告
val_fold_reports, test_fold_reports = get_report_perfold(exps_dir)

# 计算每个run的统计数据
# val_fold_stats = compute_fold_stats(val_fold_reports)
test_fold_stats = compute_fold_stats(test_fold_reports)

# 打印每个run每个fold的结果
print("\nTest Performance Per Fold:")
for run_dir, fold_results in test_fold_reports.items():
    print(f"\nRun: {run_dir}")
    for fold_result in fold_results:
        fold = fold_result['fold']
        report = fold_result['report']
        print(f"  Fold: {fold}, Accuracy: {report['accuracy']:.4f}, F1 (PD): {report['PD']['f1-score']:.4f}")
    
    # 打印该run的所有fold的均值和标准差
    stats = test_fold_stats[run_dir]
    # 打印该run的所有fold的均值和标准差
    print(f"  Mean Accuracy: {stats['accuracy']['mean']:.4f} ± {stats['accuracy']['std']:.4f}")
    print(f"  Mean F1 (PD): {stats['f1']['mean']:.4f} ± {stats['f1']['std']:.4f}")
    print(f"  Mean Precision (PD): {stats['precision']['mean']:.4f} ± {stats['precision']['std']:.4f}")
    print(f"  Mean Recall (PD): {stats['recall']['mean']:.4f} ± {stats['recall']['std']:.4f}")
    print(f"  Mean Sensitivity: {stats['sensitivity']['mean']:.4f} ± {stats['sensitivity']['std']:.4f}")
    print(f"  Mean Specificity: {stats['specificity']['mean']:.4f} ± {stats['specificity']['std']:.4f}")
# 计算所有runs的平均性能指标和标准差
    print("\n=== Average Performance Across All Test Runs ===")
    all_run_accuracies = [stats['accuracy']['mean'] for stats in test_fold_stats.values()]
    all_run_f1s = [stats['f1']['mean'] for stats in test_fold_stats.values()]
    all_run_precisions = [stats['precision']['mean'] for stats in test_fold_stats.values()]
    all_run_recalls = [stats['recall']['mean'] for stats in test_fold_stats.values()]
    all_run_sensitivities = [stats['sensitivity']['mean'] for stats in test_fold_stats.values()]
    all_run_specificities = [stats['specificity']['mean'] for stats in test_fold_stats.values()]

    # 计算所有runs的标准差的平均值
    all_run_accuracies_stds = [stats['accuracy']['std'] for stats in test_fold_stats.values()]
    all_run_f1s_stds = [stats['f1']['std'] for stats in test_fold_stats.values()]
    all_run_precisions_stds = [stats['precision']['std'] for stats in test_fold_stats.values()]
    all_run_recalls_stds = [stats['recall']['std'] for stats in test_fold_stats.values()]
    all_run_sensitivities_stds = [stats['sensitivity']['std'] for stats in test_fold_stats.values()]
    all_run_specificities_stds = [stats['specificity']['std'] for stats in test_fold_stats.values()]
    # 汇总统计部分也需要加入AUC
    all_run_aucs = [stats['auc']['mean'] for stats in test_fold_stats.values()]
    all_run_aucs_stds = [stats['auc']['std'] for stats in test_fold_stats.values()]

    # 打印所有runs的平均性能
    print(f"Mean Accuracy across runs: {np.mean(all_run_accuracies):.4f} ± {np.std(all_run_accuracies):.4f}")
    print(f"Mean F1 (PD) across runs: {np.mean(all_run_f1s):.4f} ± {np.std(all_run_f1s):.4f}")
    print(f"Mean Precision (PD) across runs: {np.mean(all_run_precisions):.4f} ± {np.std(all_run_precisions):.4f}")
    print(f"Mean Recall (PD) across runs: {np.mean(all_run_recalls):.4f} ± {np.std(all_run_recalls):.4f}")
    print(f"Mean Sensitivity across runs: {np.mean(all_run_sensitivities):.4f} ± {np.std(all_run_sensitivities):.4f}")
    print(f"Mean Specificity across runs: {np.mean(all_run_specificities):.4f} ± {np.std(all_run_specificities):.4f}")
    print(f"Mean AUC across runs: {np.mean(all_run_aucs):.4f} ± {np.std(all_run_aucs):.4f}")


    print("\n=== Compact Statistics ===")
    print("Accuracy & F1 (PD) & Precision (PD) & Recall (PD) & AUC & Sensitivity & Specificity")
    print(f"{np.mean(all_run_accuracies):.4f} ± {np.mean(all_run_accuracies_stds):.4f} & {np.mean(all_run_f1s):.4f} ± {np.mean(all_run_f1s_stds):.4f} & {np.mean(all_run_precisions):.4f} ± {np.mean(all_run_precisions_stds):.4f} & {np.mean(all_run_recalls):.4f} ± {np.mean(all_run_recalls_stds):.4f} & {np.mean(all_run_aucs):.4f} ± {np.mean(all_run_aucs_stds):.4f} & {np.mean(all_run_sensitivities):.4f} ± {np.mean(all_run_sensitivities_stds):.4f} & {np.mean(all_run_specificities):.4f} ± {np.mean(all_run_specificities_stds):.4f}")



Test Performance Per Fold:

Run: seed12
  Fold: fold_0, Accuracy: 0.8543, F1 (PD): 0.8543
  Fold: fold_1, Accuracy: 0.6700, F1 (PD): 0.6765
  Fold: fold_2, Accuracy: 0.7980, F1 (PD): 0.8113
  Fold: fold_3, Accuracy: 0.7750, F1 (PD): 0.7847
  Fold: fold_4, Accuracy: 0.7850, F1 (PD): 0.7882
  Mean Accuracy: 0.7765 ± 0.0599
  Mean F1 (PD): 0.7830 ± 0.0588
  Mean Precision (PD): 0.7665 ± 0.0823
  Mean Recall (PD): 0.8094 ± 0.0855
  Mean Sensitivity: 0.8094 ± 0.0855
  Mean Specificity: 0.7510 ± 0.0904

=== Average Performance Across All Test Runs ===
Mean Accuracy across runs: 0.7718 ± 0.0077
Mean F1 (PD) across runs: 0.7765 ± 0.0097
Mean Precision (PD) across runs: 0.7662 ± 0.0086
Mean Recall (PD) across runs: 0.7988 ± 0.0148
Mean Sensitivity across runs: 0.7988 ± 0.0148
Mean Specificity across runs: 0.7531 ± 0.0109
Mean AUC across runs: 0.8582 ± 0.0056

=== Compact Statistics ===
Accuracy & F1 (PD) & Precision (PD) & Recall (PD) & AUC & Sensitivity & Specificity
0.7718 ± 0.0544 & 0.7765 

In [5]:
import scipy.stats as stats
from collections import defaultdict

def compare_two_exps_with_wilcoxon(exps_dir1, exps_dir2):
    """
    Compare results from two experiment directories using Wilcoxon signed-rank test
    
    Args:
        exps_dir1: First experiment directory path
        exps_dir2: Second experiment directory path
        
    Returns:
        Dictionary with p-values for each metric
    """
    # Get reports for both experiment directories
    _, test_fold_reports1 = get_report_perfold(exps_dir1)
    _, test_fold_reports2 = get_report_perfold(exps_dir2)
    
    # Store metrics per fold for both experiments
    metrics_by_fold1 = defaultdict(lambda: defaultdict(list))
    metrics_by_fold2 = defaultdict(lambda: defaultdict(list))
    
    # Process first experiment directory
    for run_dir, fold_results in test_fold_reports1.items():
        for fold_result in fold_results:
            fold = fold_result['fold']
            # Store metrics for this fold
            metrics_by_fold1[fold]['accuracy'].append(fold_result['report']['accuracy'])
            metrics_by_fold1[fold]['f1'].append(fold_result['report']['PD']['f1-score'])
            metrics_by_fold1[fold]['precision'].append(fold_result['report']['PD']['precision'])
            metrics_by_fold1[fold]['recall'].append(fold_result['report']['PD']['recall'])
            metrics_by_fold1[fold]['sensitivity'].append(fold_result['report']['PD']['recall'])
            metrics_by_fold1[fold]['specificity'].append(fold_result['report']['HC']['recall'])
            metrics_by_fold1[fold]['auc'].append(fold_result['auc'])
    
    # Process second experiment directory
    for run_dir, fold_results in test_fold_reports2.items():
        for fold_result in fold_results:
            fold = fold_result['fold']
            # Store metrics for this fold
            metrics_by_fold2[fold]['accuracy'].append(fold_result['report']['accuracy'])
            metrics_by_fold2[fold]['f1'].append(fold_result['report']['PD']['f1-score'])
            metrics_by_fold2[fold]['precision'].append(fold_result['report']['PD']['precision'])
            metrics_by_fold2[fold]['recall'].append(fold_result['report']['PD']['recall'])
            metrics_by_fold2[fold]['sensitivity'].append(fold_result['report']['PD']['recall'])
            metrics_by_fold2[fold]['specificity'].append(fold_result['report']['HC']['recall'])
            metrics_by_fold2[fold]['auc'].append(fold_result['auc'])
    
    # Calculate mean values for each fold
    metrics_to_test = ['accuracy', 'f1', 'precision', 'sensitivity', 'specificity', 'auc']
    fold_means1 = defaultdict(list)
    fold_means2 = defaultdict(list)
    
    # Make sure we only use folds that exist in both experiments
    common_folds = set(metrics_by_fold1.keys()) & set(metrics_by_fold2.keys())
    
    # Calculate means for each fold
    for fold in common_folds:
        for metric in metrics_to_test:
            if metrics_by_fold1[fold][metric] and metrics_by_fold2[fold][metric]:
                fold_means1[metric].append(np.mean(metrics_by_fold1[fold][metric]))
                fold_means2[metric].append(np.mean(metrics_by_fold2[fold][metric]))
    
    # print(f"Number of common folds: {len(common_folds)}")
    # print(f"Metrics to test: {metrics_to_test}")
    # print(f"Fold means for experiment 1: {fold_means1}")
    # print(f"Fold means for experiment 2: {fold_means2}")
    
    # Run Wilcoxon signed-rank test for each metric
    results = {}
    for metric in metrics_to_test:
        if len(fold_means1[metric]) >= 5:  # Need at least 5 pairs for reliable results
            statistic, p_value = stats.wilcoxon(fold_means1[metric], fold_means2[metric])
            results[metric] = {
                'p_value': p_value,
                'significant': p_value < 0.05,
                'means1': np.mean(fold_means1[metric]),
                'means2': np.mean(fold_means2[metric]),
                'difference': np.mean(fold_means1[metric]) - np.mean(fold_means2[metric])
            }
    
    return results

# Example usage
exps_dir1 = "/home/yzhong/gits/interpretable-pd/exps/pcgita_splits_10foldnew/cross_full/allsent/"
exps_dir2 = "/home/yzhong/gits/interpretable-pd/exps/pcgita_splits_10foldnew/cross_full_fix_value/allsent/"


results = compare_two_exps_with_wilcoxon(exps_dir1, exps_dir2)

# Print results in a nice format
print(f"Statistical comparison between: \n- M{exps_dir1.strip('/').split('/')[-1]} and \n- M{exps_dir2.strip('/').split('/')[-1]}\n")
print("Wilcoxon Signed-Rank Test Results:")
for metric, result in results.items():
    significance = "* significant *" if result['significant'] else "not significant"
    direction = ">" if result['difference'] > 0 else "<"
    print(f"{metric.upper()}: {result['means1']:.4f} {direction} {result['means2']:.4f}, p={result['p_value']:.4f} ({significance})")

Statistical comparison between: 
- Mallsent and 
- Mallsent

Wilcoxon Signed-Rank Test Results:
ACCURACY: 0.7787 > 0.7601, p=0.0273 (* significant *)
F1: 0.7748 > 0.7588, p=0.0137 (* significant *)
PRECISION: 0.7923 > 0.7678, p=0.1602 (not significant)
SENSITIVITY: 0.7652 > 0.7562, p=0.2754 (not significant)
SPECIFICITY: 0.7924 > 0.7641, p=0.3223 (not significant)
AUC: 0.8301 > 0.8226, p=0.2754 (not significant)


In [6]:

def get_avg_F1_for_allfolds(exps_dir1):
    """
    Compare results from two experiment directories using Wilcoxon signed-rank test
    
    Args:
        exps_dir1: First experiment directory path
        exps_dir2: Second experiment directory path
        
    Returns:
        Dictionary with p-values for each metric
    """
    
    # print(f"Processing experiment directory: {exps_dir1}")
    # Get reports for both experiment directories
    _, test_fold_reports1 = get_report_perfold(exps_dir1)
    
    # Store metrics per fold for both experiments
    metrics_by_fold1 = defaultdict(lambda: defaultdict(list))
    
    # Process first experiment directory
    for run_dir, fold_results in test_fold_reports1.items():
        for fold_result in fold_results:
            fold = fold_result['fold']
            # Store metrics for this fold
            metrics_by_fold1[fold]['accuracy'].append(fold_result['report']['accuracy'])
            metrics_by_fold1[fold]['f1'].append(fold_result['report']['PD']['f1-score'])
            metrics_by_fold1[fold]['precision'].append(fold_result['report']['PD']['precision'])
            metrics_by_fold1[fold]['recall'].append(fold_result['report']['PD']['recall'])
            metrics_by_fold1[fold]['sensitivity'].append(fold_result['report']['PD']['recall'])
            metrics_by_fold1[fold]['specificity'].append(fold_result['report']['HC']['recall'])
            metrics_by_fold1[fold]['auc'].append(fold_result['auc'])
    
    
    # Calculate mean values for each fold
    metrics_to_test = ['accuracy', 'f1', 'precision', 'sensitivity', 'specificity', 'auc']
    fold_means1 = defaultdict(list)
    
    # Make sure we only use folds that exist in both experiments
    common_folds = set(metrics_by_fold1.keys()) 
    fold_means1_f1 = []
    # Calculate means for each fold
    for i, fold in enumerate(common_folds):

        if metrics_by_fold1[fold]['f1'] :
            fold_means1_f1.append([i+1, np.mean(metrics_by_fold1[fold]['f1'])])
            
    return fold_means1_f1

exp1 = "cross_full"
exp2 = "cross_full_fix"
exp3 = "cross_full_fix_value"
exp4 = "cross_token_oldcate"
exps = [exp1, exp2, exp3, exp4]

def get_report_perexp(exp):
    
    dir1 = "/home/yzhong/gits/interpretable-pd/exps/pcgita_splits_10foldnew/"
    dir2 = "/home/yzhong/gits/interpretable-pd/exps/gita/"
    dir3 = "/home/yzhong/gits/interpretable-pd/exps/gita-splitmono/"
    pertask2f1 = defaultdict(list)

    exps_dir1 = os.path.join(dir1, exp, 'allsent') 
    fold_means_allsent = get_avg_F1_for_allfolds(exps_dir1)
    pertask2f1['ALLSENT'] = fold_means_allsent
    
    exps_dir3 = os.path.join(dir3, exp, 'MONOLOGUE')
    fold_means_splitmono = get_avg_F1_for_allfolds(exps_dir3)
    pertask2f1['SPLIT-MONO'] = fold_means_splitmono

    tasks = ["DDK", "READ", "SENTENCES", "SUSTAINED-VOWELS", "WORDS"]
    for task in tasks:
        exp_dir = os.path.join(dir2, exp, task)
        if os.path.exists(exp_dir):
            fold_means = get_avg_F1_for_allfolds(exp_dir)
            pertask2f1[task] = fold_means
            
    

    return pertask2f1


pertask2f1_1 = get_report_perexp(exp1)
pertask2f1_2 = get_report_perexp(exp2)
pertask2f1_3 = get_report_perexp(exp3)
pertask2f1_4 = get_report_perexp(exp4)



In [9]:
import pandas as pd
import numpy as np

def convert_nested_dict_to_dataframe(data_dict, modelid="M1"):
    """
    Convert a nested defaultdict to a pandas DataFrame
    
    Args:
        data_dict: Nested defaultdict with task names as keys and lists of [fold_number, f1_score] as values
        
    Returns:
        DataFrame with tasks, fold numbers, and F1 scores in long format
    """
    # Create lists to store the data in long format
    tasks = []
    fold_numbers = []
    f1_scores = []
    
    # Process each task
    taskidx = 0
    for task, fold_scores in data_dict.items():
        for fold_num, score in fold_scores:
            tasks.append(task)
            fold_numbers.append(int(fold_num) + taskidx * 10)  # Adjust fold number to be unique across tasks
            f1_scores.append(score)
        taskidx += 1  # Increment task index to adjust fold numbers
    
    # Create DataFrame in long format
    df = pd.DataFrame({
        'Task': tasks,
        'Fold': fold_numbers,
        'F1_Score': f1_scores,
        'ModelID': modelid
    })
    
    # Sort by task and fold
    df = df.sort_values(by=['Task', 'Fold']).reset_index(drop=True)
    
    # Add a sequential ID column
    # df['ID'] = range(1, len(df) + 1)
    
    # Format the DataFrame for better display
    pd.options.display.float_format = '{:.4f}'.format
    
    return df

# Example usage
df1 = convert_nested_dict_to_dataframe(pertask2f1_1, modelid="M1")
df2 = convert_nested_dict_to_dataframe(pertask2f1_2, modelid="M2")
df3 = convert_nested_dict_to_dataframe(pertask2f1_3, modelid="M3")
df4 = convert_nested_dict_to_dataframe(pertask2f1_4, modelid="M4")


df_all = pd.concat([df1, df2, df3, df4], ignore_index=True)

# Print the combined dataframe
print("Combined DataFrame:")
print(df_all[:50])



Combined DataFrame:
                Task  Fold  F1_Score ModelID
0            ALLSENT     1    0.7673      M1
1            ALLSENT     2    0.8066      M1
2            ALLSENT     3    0.8320      M1
3            ALLSENT     4    0.8796      M1
4            ALLSENT     5    0.8073      M1
5            ALLSENT     6    0.8621      M1
6            ALLSENT     7    0.6553      M1
7            ALLSENT     8    0.7682      M1
8            ALLSENT     9    0.6336      M1
9            ALLSENT    10    0.7358      M1
10               DDK    21    0.6996      M1
11               DDK    22    0.8127      M1
12               DDK    23    0.7864      M1
13               DDK    24    0.8372      M1
14               DDK    25    0.7137      M1
15              READ    31    0.7487      M1
16              READ    32    0.8189      M1
17              READ    33    0.5260      M1
18              READ    34    0.8692      M1
19              READ    35    0.8405      M1
20         SENTENCES    41    0.624

In [8]:

import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.anova import anova_lm
from itertools import combinations
import matplotlib.pyplot as plt
import seaborn as sns

def fit_lme_model(df):
    """
    Fit a Linear Mixed Effects model with F1_Score as response variable,
    ModelID as fixed effect, and Task and Fold as random effects.
    
    Args:
        df: DataFrame containing the data with columns 'Task', 'Fold', 'F1_Score', and 'ModelID'
        
    Returns:
        fitted model and summary
    """
    # Fit the LME model
    # Using Treatment contrast with M1 as reference level
    model = smf.mixedlm("F1_Score ~ ModelID", df, 
                         groups=df["Task"],
                         re_formula="~Fold")
    
    result = model.fit()
    return model, result

def run_pairwise_comparisons(df):
    """
    Run pairwise comparisons between all model combinations
    
    Args:
        df: DataFrame containing the data
        
    Returns:
        DataFrame with pairwise comparison results
    """
    # Get unique model IDs
    models = df['ModelID'].unique()
    
    # Create empty lists to store comparison results
    pair_list = []
    diff_list = []
    pvalue_list = []
    significant_list = []
    
    # Iterate through all pairs of models
    for m1, m2 in combinations(models, 2):
        # Subset data for the two models
        data_m1 = df[df['ModelID'] == m1]['F1_Score']
        data_m2 = df[df['ModelID'] == m2]['F1_Score']
        
        # Calculate mean difference
        mean_diff = data_m1.mean() - data_m2.mean()
        
        # Fit a model for just these two models
        subset_df = df[df['ModelID'].isin([m1, m2])].copy()
        subset_df['ModelID'] = pd.Categorical(subset_df['ModelID'], 
                                             categories=[m1, m2], 
                                             ordered=False)
        
        model = smf.mixedlm("F1_Score ~ ModelID", subset_df, 
                           groups=subset_df["Task"],
                           re_formula="~Fold")
        result = model.fit()
        
        # Get p-value for the model comparison
        p_value = result.pvalues['ModelID[T.{}]'.format(m2)]
        significant = p_value < 0.05
        
        # Append results
        pair_list.append(f"{m1} vs {m2}")
        diff_list.append(mean_diff)
        pvalue_list.append(p_value)
        significant_list.append(significant)
    
    # Create comparison results DataFrame
    comparison_df = pd.DataFrame({
        'Comparison': pair_list,
        'Mean_Difference': diff_list,
        'P_Value': pvalue_list,
        'Significant': significant_list
    })
    
    return comparison_df.sort_values('P_Value')

def visualize_results(df, lme_result, comparison_df):
    """
    Create visualizations of the model results
    
    Args:
        df: Original DataFrame
        lme_result: Fitted LME model result
        comparison_df: DataFrame with pairwise comparison results
    """
    # Set up the figure
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # 1. Box plot of F1 scores by model
    sns.boxplot(x='ModelID', y='F1_Score', data=df, ax=ax1)
    ax1.set_title('Distribution of F1 Scores by Model')
    ax1.set_xlabel('Model')
    ax1.set_ylabel('F1 Score')
    
    # Add mean F1 score labels
    model_means = df.groupby('ModelID')['F1_Score'].mean()
    for i, model in enumerate(model_means.index):
        ax1.text(i, model_means[model] + 0.02, f'{model_means[model]:.4f}', 
                ha='center', va='bottom', fontweight='bold')
    
    # 2. Heatmap of p-values for model comparisons
    # Create a matrix of p-values
    models = df['ModelID'].unique()
    p_matrix = pd.DataFrame(np.nan, index=models, columns=models)
    
    for _, row in comparison_df.iterrows():
        m1, m2 = row['Comparison'].split(' vs ')
        p_matrix.loc[m1, m2] = row['P_Value']
        p_matrix.loc[m2, m1] = row['P_Value']  # Symmetric matrix
    
    # Plot heatmap with p-values
    mask = np.triu(np.ones_like(p_matrix, dtype=bool))
    sns.heatmap(p_matrix, annot=True, fmt='.4f', cmap='viridis', 
               mask=mask, ax=ax2)
    ax2.set_title('P-values for Pairwise Model Comparisons')
    
    plt.tight_layout()
    plt.savefig('/home/yzhong/gits/interpretable-pd/scripts/evaluation/lme_model_comparison.png', dpi=300)
    plt.show()
    
    # Also create a task-specific visualization
    plt.figure(figsize=(12, 8))
    sns.boxplot(x='Task', y='F1_Score', hue='ModelID', data=df)
    plt.title('F1 Scores by Task and Model')
    plt.xlabel('Task')
    plt.ylabel('F1 Score')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig('/home/yzhong/gits/interpretable-pd/scripts/evaluation/task_model_comparison.png', dpi=300)
    plt.show()

# def main():
#     # Load data from the four models
#     df1 = pd.read_csv('/home/yzhong/gits/interpretable-pd/splits/model1_results.csv')
#     df2 = pd.read_csv('/home/yzhong/gits/interpretable-pd/splits/model2_results.csv')
#     df3 = pd.read_csv('/home/yzhong/gits/interpretable-pd/splits/model3_results.csv')
#     df4 = pd.read_csv('/home/yzhong/gits/interpretable-pd/splits/model4_results.csv')
    
    # If the CSV files don't exist, we can recreate the DataFrame from the notebook
    # This is just placeholder code - in practice you would have saved the DataFrame
    # df1 = convert_nested_dict_to_dataframe(pertask2f1_1, modelid="M1")
    # df2 = convert_nested_dict_to_dataframe(pertask2f1_2, modelid="M2")
    # df3 = convert_nested_dict_to_dataframe(pertask2f1_3, modelid="M3")
    # df4 = convert_nested_dict_to_dataframe(pertask2f1_4, modelid="M4")
    
    # Combine all dataframes
# df_all = pd.concat([df1, df2, df3, df4], ignore_index=True)

# Fit LME model
model, result = fit_lme_model(df_all)

# Print model summary
print("=" * 80)
print("Linear Mixed Effects Model Summary")
print("=" * 80)
print(result.summary())

# Run pairwise comparisons
comparison_df = run_pairwise_comparisons(df_all)

# Print comparison results
print("\n" + "=" * 80)
print("Pairwise Model Comparisons")
print("=" * 80)
print(comparison_df.to_string(index=False))

# Create visualizations
visualize_results(df_all, result, comparison_df)

# Provide interpretation of results
print("\n" + "=" * 80)
print("Interpretation of Results")
print("=" * 80)

# Compare mean F1 scores
model_means = df_all.groupby('ModelID')['F1_Score'].mean().sort_values(ascending=False)
print("Models ranked by mean F1 score:")
for model, mean in model_means.items():
    print(f"{model}: {mean:.4f}")

# Summarize significant differences
significant_comparisons = comparison_df[comparison_df['Significant']]
print("\nSignificant differences between models:")
if len(significant_comparisons) == 0:
    print("No statistically significant differences were found between any models.")
else:
    for _, row in significant_comparisons.iterrows():
        better_model = row['Comparison'].split(' vs ')[0] if row['Mean_Difference'] > 0 else row['Comparison'].split(' vs ')[1]
        print(f"- {row['Comparison']}: p={row['P_Value']:.4f}, {better_model} performs better")



/data/storage500/yzhongenv/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/data/storage500/yzhongenv/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  warnings.warn(
/data/storage500/yzhongenv/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


LinAlgError: Singular matrix

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from itertools import combinations

def run_lme_analysis(df):
    """
    Run Linear Mixed-Effects model on F1 scores with ModelID as fixed effect,
    random intercepts and slopes for ModelID by Task, and random intercepts for Task:Fold
    
    Args:
        df: DataFrame with columns Task, Fold, F1_Score, ModelID
        
    Returns:
        Dictionary with pairwise comparison results
    """
    # Ensure categorical variables are properly coded
    df['ModelID'] = pd.Categorical(df['ModelID'])
    df['Task'] = pd.Categorical(df['Task'])
    df['Fold'] = pd.Categorical(df['Fold'])
    
    # Create Task:Fold interaction term
    df['Task_Fold'] = df['Task'].astype(str) + ':' + df['Fold'].astype(str)
    
    print("Fitting complex mixed model with random slopes...")
    
    # Fit the full model with all data
    try:
        # The re_formula specifies random effects structure
        # "1 + ModelID" means random intercepts and slopes for ModelID
        model = smf.mixedlm("F1_Score ~ ModelID", 
                           data=df, 
                           groups=df["Task"],
                           re_formula="1 + ModelID")
        
        result = model.fit(reml=True)
        print("\nModel summary:")
        print(result.summary())
        
        # Store all model coefficients
        coefficients = result.params
        p_values = result.pvalues
        
        # Get all model pairs for comparison
        model_ids = sorted(df['ModelID'].unique())
        model_pairs = list(combinations(model_ids, 2))
        
        # Run pairwise comparisons between models
        print("\nPairwise comparisons between models:")
        print("-" * 60)
        print(f"{'Comparison':15s} | {'Estimate':10s} | {'p-value':10s} | Result")
        print("-" * 60)
        
        results_dict = {}
        
        for m1, m2 in model_pairs:
            # Create subset of data with just these two models
            subset_df = df[df['ModelID'].isin([m1, m2])].copy()
            subset_df['ModelID'] = pd.Categorical(subset_df['ModelID'], 
                                                 categories=[m1, m2],
                                                 ordered=False)
            
            # Refit model on subset with the same complex random effects structure
            # This matches the R formula: F1 ~ Model + (1 + Model | task) + (1 | task:fold)
            subset_model = smf.mixedlm("F1_Score ~ ModelID", 
                                      data=subset_df, 
                                      groups=subset_df["Task"],
                                      re_formula="1 + ModelID")
            
            subset_result = subset_model.fit(reml=True)
            
            # Get the coefficient for the second model (compared to first as reference)
            model_param = f"ModelID[T.{m2}]"
            if model_param in subset_result.params:
                model_effect = subset_result.params[model_param]
                p_value = subset_result.pvalues[model_param]
                
                significant = p_value < 0.05
                significance_marker = "* significant *" if significant else "not significant"
                
                # Calculate mean F1 scores for comparison
                mean1 = subset_df[subset_df['ModelID'] == m1]['F1_Score'].mean()
                mean2 = subset_df[subset_df['ModelID'] == m2]['F1_Score'].mean()
                
                print(f"{m1} vs {m2:5s} | {model_effect:.6f} | {p_value:.6f} | {significance_marker}")
                
                # Store result
                results_dict[f"{m1}_vs_{m2}"] = {
                    'estimate': model_effect,
                    'p_value': p_value,
                    'significant': significant,
                    'mean_f1_first': mean1,
                    'mean_f1_second': mean2
                }
            else:
                print(f"Warning: Could not find parameter for {m1} vs {m2} comparison")
        
        # Print overall comparison table
        print("\n")
        print("=" * 80)
        print("Summary of Model Comparisons")
        print("=" * 80)
        print(f"{'Model':8s} | {'Mean F1':10s} | {'Significantly better than':30s}")
        print("-" * 80)
        
        # For each model, list which models it's significantly better than
        for model_id in model_ids:
            mean_f1 = df[df['ModelID'] == model_id]['F1_Score'].mean()
            better_than = []
            worse_than = []
            
            for pair, result in results_dict.items():
                m1, m2 = pair.split('_vs_')
                if m1 == model_id and result['significant']:
                    if result['estimate'] < 0:  # negative estimate means m2 is better
                        worse_than.append(m2)
                    else:
                        better_than.append(m2)
                elif m2 == model_id and result['significant']:
                    if result['estimate'] > 0:  # positive estimate means m2 is better
                        worse_than.append(m1)
                    else:
                        better_than.append(m1)
            
            better_than_str = ', '.join(better_than) if better_than else "none"
            worse_than_str = ', '.join(worse_than) if worse_than else "none"
            print(f"{model_id:8s} | {mean_f1:.6f} | Better than: {better_than_str}")
            print(f"{' ':8s} | {' ':10s} | Worse than: {worse_than_str}")
        
        return results_dict
    
    except Exception as e:
        print(f"Error fitting model: {e}")
        import traceback
        traceback.print_exc()
        return None

# Run the analysis on the combined dataframe
results = run_lme_analysis(df_all)


Fitting complex mixed model with random slopes...


/data/storage500/yzhongenv/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/data/storage500/yzhongenv/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  warnings.warn(
/data/storage500/yzhongenv/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/data/storage500/yzhongenv/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2261: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)



Model summary:
                   Mixed Linear Model Regression Results
Model:                    MixedLM        Dependent Variable:        F1_Score
No. Observations:         160            Method:                    REML    
No. Groups:               7              Scale:                     0.0047  
Min. group size:          20             Log-Likelihood:            179.7828
Max. group size:          40             Converged:                 Yes     
Mean group size:          22.9                                              
----------------------------------------------------------------------------
                                  Coef.  Std.Err.   z    P>|z| [0.025 0.975]
----------------------------------------------------------------------------
Intercept                          0.749    0.019 39.545 0.000  0.712  0.786
ModelID[T.M2]                      0.008    0.018  0.418 0.676 -0.028  0.043
ModelID[T.M3]                     -0.079    0.022 -3.542 0.000 -0.123 -0.035
Mod

/data/storage500/yzhongenv/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/data/storage500/yzhongenv/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  warnings.warn(
/data/storage500/yzhongenv/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/data/storage500/yzhongenv/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2261: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)
/data/storage500/yzhongenv/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: 

M1 vs M2    | 0.007375 | 0.690093 | not significant
M1 vs M3    | -0.078344 | 0.000071 | * significant *


/data/storage500/yzhongenv/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/data/storage500/yzhongenv/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2261: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)
/data/storage500/yzhongenv/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/data/storage500/yzhongenv/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/data/storage500/yzhongenv/lib/python3.10/site-packages/statsmodels/r

M1 vs M4    | -0.017082 | 0.405194 | not significant
M2 vs M3    | -0.087544 | 0.000767 | * significant *
M2 vs M4    | -0.024467 | 0.119183 | not significant
M3 vs M4    | 0.061084 | 0.001532 | * significant *


Summary of Model Comparisons
Model    | Mean F1    | Significantly better than     
--------------------------------------------------------------------------------
M1       | 0.752684 | Better than: none
         |            | Worse than: M3
M2       | 0.761428 | Better than: none
         |            | Worse than: M3
M3       | 0.678999 | Better than: M1, M2, M4
         |            | Worse than: none
M4       | 0.736469 | Better than: none
         |            | Worse than: M3


/data/storage500/yzhongenv/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from itertools import combinations

def run_lme_analysis(df):
    """
    Run Linear Mixed-Effects model on F1 scores with ModelID as fixed effect,
    random intercepts and slopes for ModelID by Task, and random intercepts for Task:Fold
    
    Args:
        df: DataFrame with columns Task, Fold, F1_Score, ModelID
        
    Returns:
        Dictionary with pairwise comparison results
    """
    # Ensure categorical variables are properly coded
    df['ModelID'] = pd.Categorical(df['ModelID'])
    df['Task'] = pd.Categorical(df['Task'])
    df['Fold'] = pd.Categorical(df['Fold'])
    
    # Create Task:Fold interaction term for the nested random effect
    df['Task_Fold'] = df['Task'].astype(str) + '_' + df['Fold'].astype(str)
    df['Task_Fold'] = pd.Categorical(df['Task_Fold'])
    
    print("Fitting complex mixed model with random slopes and nested random effects...")
    
    # Approach: We'll use the Task as the main grouping variable
    # And include both ModelID (for random slopes) and Task_Fold in the re_formula
    
    try:
        # First, we'll try with statsmodels' approach to nested random effects
        # Task is the main grouping, ModelID is the random slope, Task_Fold is nested random effect
        formula = "F1_Score ~ ModelID"
        
        # Using vc_formula to add the task:fold nested random effect
        vc_formulas = {"Task_Fold": "0 + C(Task_Fold)"}
        
        model = smf.mixedlm(formula, 
                           data=df, 
                           groups=df["Task"],
                           re_formula="1 + ModelID",
                           vc_formula=vc_formulas)
        
        result = model.fit(reml=True)
        print("\nModel summary:")
        print(result.summary())
        
        # Get all model pairs for comparison
        model_ids = sorted(df['ModelID'].unique())
        model_pairs = list(combinations(model_ids, 2))
        
        # Run pairwise comparisons between models
        print("\nPairwise comparisons between models:")
        print("-" * 60)
        print(f"{'Comparison':15s} | {'Estimate':10s} | {'p-value':10s} | Result")
        print("-" * 60)
        
        results_dict = {}
        
        for m1, m2 in model_pairs:
            # Create subset of data with just these two models
            subset_df = df[df['ModelID'].isin([m1, m2])].copy()
            subset_df['ModelID'] = pd.Categorical(subset_df['ModelID'], 
                                                 categories=[m1, m2],
                                                 ordered=False)
            
            # Refit model on subset with both random effects
            subset_vc_formulas = {"Task_Fold": "0 + C(Task_Fold)"}
            
            subset_model = smf.mixedlm("F1_Score ~ ModelID", 
                                      data=subset_df, 
                                      groups=subset_df["Task"],
                                      re_formula="1 + ModelID",
                                      vc_formula=subset_vc_formulas)
            
            subset_result = subset_model.fit(reml=True)
            
            # Get the coefficient for the comparison
            model_param = f"ModelID[T.{m2}]"
            if model_param in subset_result.params:
                model_effect = subset_result.params[model_param]
                p_value = subset_result.pvalues[model_param]
                
                significant = p_value < 0.05
                significance_marker = "* significant *" if significant else "not significant"
                
                # Calculate mean F1 scores for reference
                mean1 = subset_df[subset_df['ModelID'] == m1]['F1_Score'].mean()
                mean2 = subset_df[subset_df['ModelID'] == m2]['F1_Score'].mean()
                
                print(f"{m1} vs {m2:5s} | {model_effect:.6f} | {p_value:.6f} | {significance_marker}")
                
                # Store result
                results_dict[f"{m1}_vs_{m2}"] = {
                    'estimate': model_effect,
                    'p_value': p_value,
                    'significant': significant,
                    'mean_f1_first': mean1,
                    'mean_f1_second': mean2
                }
            else:
                print(f"Warning: Could not find parameter for {m1} vs {m2} comparison")
        
        # Print overall comparison table
        print("\n")
        print("=" * 80)
        print("Summary of Model Comparisons")
        print("=" * 80)
        print(f"{'Model':8s} | {'Mean F1':10s} | {'Better than':30s} | {'Worse than':30s}")
        print("-" * 80)
        
        # For each model, list which models it's significantly better/worse than
        for model_id in model_ids:
            mean_f1 = df[df['ModelID'] == model_id]['F1_Score'].mean()
            better_than = []
            worse_than = []
            
            for pair, result in results_dict.items():
                m1, m2 = pair.split('_vs_')
                if m1 == model_id and result['significant']:
                    if result['estimate'] < 0:  # negative estimate means m2 is better
                        worse_than.append(m2)
                    else:  # positive estimate means m1 is better
                        better_than.append(m2)
                elif m2 == model_id and result['significant']:
                    if result['estimate'] > 0:  # positive estimate means m1 is better
                        worse_than.append(m1)
                    else:  # negative estimate means m2 is better
                        better_than.append(m1)
            
            better_than_str = ', '.join(better_than) if better_than else "none"
            worse_than_str = ', '.join(worse_than) if worse_than else "none"
            print(f"{model_id:8s} | {mean_f1:.6f} | {better_than_str:30s} | {worse_than_str:30s}")
        
        return results_dict
    
    except Exception as e:
        print(f"Error fitting model: {e}")
        import traceback
        traceback.print_exc()
        return None
    
results = run_lme_analysis(df_all)
